In [37]:
#!pip install beautifulsoup4 requests openpyxl

In [8]:
import requests
from bs4 import BeautifulSoup
import openpyxl

In [9]:
import requests
requests.get('http://disclosures.ifc.org/enterprise-search-results-home')

<Response [200]>

In [11]:
import xml.etree.ElementTree as ET
# URL of the website
url = "http://disclosures.ifc.org/enterprise-search-results-home"

# Send a GET request to the website
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')



In [24]:
from bs4 import BeautifulSoup

def extract_project_data(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    project_data = []

    # Find all projects
    projects_divs = soup.find_all("div", class_="row margin-top15 projects")

    for project_div in projects_divs:
        
        # Project name and link
        project_link_element = project_div.find("a", class_="search-head")
        project_name = project_link_element.text.strip() if project_link_element else None
        project_link = project_link_element["href"] if project_link_element else None
        
        # Country
        country_div = project_div.find("div", class_="col-3 col-sm-3 col-md-3 col-lg-3 col-xl-3 search_result_col padding0")
        if country_div:
            country = country_div.find("div", class_="spi-text padding0 esrs-value")
            country = country.text.strip() if country else None
        else:
            country = None
        
        # Append to the project_data list
        project_data.append({
            "Project Name": project_name,
            "Project Link": project_link,
            "Country": country
        })

    return project_data

# Usage example:
with open("disclosure_search.html", "r") as file:
    html_content = file.read()
    data = extract_project_data(html_content)
    for project in data:
        print(project)


{'Project Name': 'Healthspring', 'Project Link': 'https://disclosures.ifc.org/project-detail/ESRS/46672/healthspring', 'Country': 'India'}
{'Project Name': 'Healthspring', 'Project Link': 'https://disclosures.ifc.org/project-detail/SII/46672/healthspring', 'Country': 'India'}
{'Project Name': 'DCM BPI', 'Project Link': 'https://disclosures.ifc.org/project-detail/SII/47671/dcm-bpi', 'Country': 'Philippines'}
{'Project Name': 'GCap SL Bond', 'Project Link': 'https://disclosures.ifc.org/project-detail/ESRS/48231/gcap-sl-bond', 'Country': 'Georgia'}
{'Project Name': 'GCap SL Bond', 'Project Link': 'https://disclosures.ifc.org/project-detail/SII/48231/gcap-sl-bond', 'Country': 'Georgia'}
{'Project Name': 'CnC-Tembo Hydro', 'Project Link': 'https://disclosures.ifc.org/project-detail/SIVP/48075/cnc-tembo-hydro', 'Country': 'Burundi'}
{'Project Name': 'BioVac - Pfizer', 'Project Link': 'https://disclosures.ifc.org/project-detail/ESRS/46582/biovac-pfizer', 'Country': 'South Africa'}
{'Project N

In [20]:
#project_data = extract_project_data(soup)

In [25]:
import pandas as pd

project_data = pd.DataFrame(data)
project_data

,Project Name,Project Link,Country
0,Healthspring,https://disclosures.ifc.org/project-detail/ESR...,India
1,Healthspring,https://disclosures.ifc.org/project-detail/SII...,India
2,DCM BPI,https://disclosures.ifc.org/project-detail/SII...,Philippines
3,GCap SL Bond,https://disclosures.ifc.org/project-detail/ESR...,Georgia
4,GCap SL Bond,https://disclosures.ifc.org/project-detail/SII...,Georgia
5,CnC-Tembo Hydro,https://disclosures.ifc.org/project-detail/SIV...,Burundi
6,BioVac - Pfizer,https://disclosures.ifc.org/project-detail/ESR...,South Africa
7,BioVac - Pfizer,https://disclosures.ifc.org/project-detail/SII...,South Africa
8,DCM B3T2 BT,https://disclosures.ifc.org/project-detail/SII...,Romania
9,Brookfield RJ,https://disclosures.ifc.org/project-detail/ESR...,India


In [33]:
from bs4 import BeautifulSoup

def extract_project_details_from_html(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    project_details = {}

    # Extracting "Disclaimer text data"
    disclaimer_div = soup.find("div", class_="card-body disclaimer-text")
    if disclaimer_div:
        disclaimer_data = disclaimer_div.get_text(strip=True)
        project_details["Disclaimer text data"] = disclaimer_data

    # Extracting "Project Description"
    project_description_div = soup.find("app-sii-project-description")
    if project_description_div:
        project_description = project_description_div.find("div", class_="esrs-value overflow-auto remove-style")
        project_details["Project Description"] = project_description.get_text(strip=True) if project_description else None

    # Extracting "Country"
    country_value_div = soup.select_one("div.esrs-name:contains('Country') + div .esrs-value")
    if country_value_div:
        project_details["Country"] = country_value_div.get_text(strip=True)

    # Extracting other attributes
    attributes = [
        "Project Number",
        "Environmental Category",
        "Sector",
        "Company Name",
        "Region",
        "Status",
        "Industry",
        "Date ESRS Disclosed",
        "Previous Events",
        "Department",
        "Overview of IFC's Scope of Review"
    ]

    for attribute in attributes:
        # Searching for the attribute title
        title_element = soup.find("div", string=attribute)
        if title_element:
            # Extracting the associated data for the attribute
            data_element = title_element.find_next("div")
            project_details[attribute] = data_element.get_text(strip=True) if data_element else None
        else:
            project_details[attribute] = None
    
    return project_details

# Usage example:
with open("DCM-BPI.html", "r", encoding="utf-8") as file:
    html_content = file.read()
    data = extract_project_details_from_html(html_content)

In [36]:
project_metadata = pd.DataFrame([data])
project_metadata

,Disclaimer text data,Project Description,Country,Project Number,Environmental Category,Sector,Company Name,Region,Status,Industry,Date ESRS Disclosed,Previous Events,Department,Overview of IFC's Scope of Review
0,This Summary of Investment Information (SII) i...,International Finance Corporation (“IFC”) is c...,Philippines,47671,FI-2,Commercial Banking - General,BANK OF THE PHILIPPINE ISLANDS,East Asia and the Pacific,Pending Disbursement,Financial Institutions,None,"Approved : Jun 20, 2023",CF4S4 - Regional Industry - FIG Asia & Pac/FIG...,None
